In [1]:
import argparse
import json
import datetime as dt
import numpy as np
from scipy.io.wavfile import write

import torch

import params
from model import GradTTS
from text import text_to_sequence, cmudict
from text.symbols import symbols
from utils import intersperse

import sys
sys.path.append('hifi/')
from env import AttrDict
from models import Generator as HiFiGAN


HIFIGAN_CONFIG = './checkpts/hifigan-config.json'
HIFIGAN_CHECKPT = './checkpts/hifigan.pt'

In [2]:
import torch
from model import monotonic_align
import numpy as np

In [58]:
neg_cent, mask = torch.load("1.pt"), torch.load("2.pt")

In [60]:
mask.shape

torch.Size([16, 379, 1356])

In [62]:
mask.sum(2)[:, 0]

tensor([ 647.,  758.,  715.,  833.,  857.,  855.,  739.,  401.,  845.,  639.,
         805., 1354.,  852.,  775.,  419.,  781.], device='cuda:0')

In [63]:
mask.sum(2).shape

torch.Size([16, 379])

In [64]:
mask.shape

torch.Size([16, 379, 1356])

In [21]:
device = neg_cent.device
dtype = neg_cent.dtype
neg_cent = neg_cent.data.cpu().numpy().astype(np.float32)
path = np.zeros(neg_cent.shape, dtype=np.int32)

t_t_max = mask.sum(1)[:, 0].data.cpu().numpy().astype(np.int32)
t_s_max = mask.sum(2)[:, 0].data.cpu().numpy().astype(np.int32)


In [22]:
t_t_max.shape, t_s_max.shape

((16,), (16,))

In [23]:
paths = np.zeros(neg_cent.shape, dtype=np.int32)
paths

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [66]:
paths.shape

(16, 427, 1236)

In [24]:
b = path.shape[0]
b, neg_cent

(16,
 array([[[-1954.3187 , -1944.3813 , -1938.3387 , ...,   -87.59668,
            -87.59668,   -87.59668],
         [-2102.042  , -2090.9995 , -2086.7498 , ...,   -79.04165,
            -79.04165,   -79.04165],
         [-2189.8872 , -2177.5435 , -2171.264  , ...,   -75.5147 ,
            -75.5147 ,   -75.5147 ],
         ...,
         [-2275.0232 , -2262.0144 , -2256.3518 , ...,   -73.51508,
            -73.51508,   -73.51508],
         [-2275.0232 , -2262.0144 , -2256.3518 , ...,   -73.51508,
            -73.51508,   -73.51508],
         [-2275.0232 , -2262.0144 , -2256.3518 , ...,   -73.51508,
            -73.51508,   -73.51508]],
 
        [[-2456.302  , -1365.0197 , -1136.9724 , ...,   -77.6333 ,
            -77.6333 ,   -77.6333 ],
         [-2420.476  , -1337.7217 , -1109.9849 , ...,   -78.31124,
            -78.31124,   -78.31124],
         [-2640.5413 , -1493.263  , -1250.2611 , ...,   -74.57619,
            -74.57619,   -74.57619],
         ...,
         [-2620.2478 , -1482

In [25]:
values = neg_cent

In [ ]:
path, neg_cent, t_t_max, t_s_max

In [70]:
a = []

for i, j in zip([path, neg_cent, t_t_max, t_s_max], range(4)):
    with open(f"{j}.pkl", "rb") as t:
        a.append(pickle.load(t))
path, neg_cent, t_t_max, t_s_max = a

In [71]:
path.shape, max(t_t_max)

((16, 427, 1236), 427)

In [73]:
for i in range(paths.shape[0]):
    path, value, t_y, t_x, max_neg_val = paths[i], neg_cent[i], t_t_max[i], t_s_max[i], -1e9
    index = t_x - 1
    for y in range(t_y):
        for x in range(max(0, t_x + y - t_y), min(t_x, y + 1)):
            if x == y:
                v_cur = max_neg_val
            else:
                v_cur = value[y-1, x]
            if x == 0:
                if y == 0:
                    v_prev = 0.
                else:
                    v_prev = max_neg_val
            else:
                v_prev = value[y-1, x-1]
            value[y, x] += max(v_prev, v_cur)

    for y in range(t_y - 1, -1, -1):
        path[y, index] = 1
        if index != 0 and (index == y or value[y-1, index] < value[y-1, index-1]):
            index = index - 1

In [69]:
y

468

In [50]:
t_s_max.sort()

In [51]:
t_t_max

array([105, 117, 129, 151, 165, 179, 187, 189, 193, 205, 205, 221, 243,
       277, 281, 379], dtype=int32)

In [52]:
index

1353

In [53]:
t_x

1354

In [42]:
for y in range(t_y - 1, -1, -1):
    print(y)

378
377
376
375
374
373
372
371
370
369
368
367
366
365
364
363
362
361
360
359
358
357
356
355
354
353
352
351
350
349
348
347
346
345
344
343
342
341
340
339
338
337
336
335
334
333
332
331
330
329
328
327
326
325
324
323
322
321
320
319
318
317
316
315
314
313
312
311
310
309
308
307
306
305
304
303
302
301
300
299
298
297
296
295
294
293
292
291
290
289
288
287
286
285
284
283
282
281
280
279
278
277
276
275
274
273
272
271
270
269
268
267
266
265
264
263
262
261
260
259
258
257
256
255
254
253
252
251
250
249
248
247
246
245
244
243
242
241
240
239
238
237
236
235
234
233
232
231
230
229
228
227
226
225
224
223
222
221
220
219
218
217
216
215
214
213
212
211
210
209
208
207
206
205
204
203
202
201
200
199
198
197
196
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
150
149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129


In [43]:
path.shape

(427, 1236)

In [45]:
index

1353

In [44]:
t_y

379

In [34]:
import pickle

In [54]:
a = []

for i, j in zip([path, neg_cent, t_t_max, t_s_max], range(4)):
    with open(f"{j}.pkl", "rb") as t:
        a.append(pickle.load(t))
path, neg_cent, t_t_max, t_s_max = a

In [55]:
path, neg_cent, t_t_max, t_s_max = a
path

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [57]:
t_s_max.sort()
t_s_max

array([ 401,  419,  639,  647,  715,  739,  758,  775,  781,  805,  833,
        845,  852,  855,  857, 1354], dtype=int32)

In [39]:
path.shape

(16, 379, 1356)

In [40]:
import model.monotonic_align.core import maximum_path_c

SyntaxError: invalid syntax (1891214979.py, line 1)

In [30]:
path.shape

(427, 1236)

In [32]:
path.mean()

0.0002936874256307648

In [33]:
torch.from_numpy(path).to(device=device, dtype=dtype)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [28]:
t_y

155

In [29]:
index

579

In [16]:
t_y

175

In [10]:
from model.monotonic_align.core import maximum_path_c

ModuleNotFoundError: No module named 'model.monotonic_align.core'

In [ ]:
  for y in range(t_y):
    for x in range(max(0, t_x + y - t_y), min(t_x, y + 1)):
      if x == y:
        v_cur = max_neg_val
      else:
        v_cur = value[y-1, x]
      if x == 0:
        if y == 0:
          v_prev = 0.
        else:
          v_prev = max_neg_val
      else:
        v_prev = value[y-1, x-1]
      value[y, x] += max(v_prev, v_cur)

  for y in range(t_y - 1, -1, -1):
    path[y, index] = 1
    if index != 0 and (index == y or value[y-1, index] < value[y-1, index-1]):
      index = index - 1

In [ ]:
maximum_path_c(path, neg_cent, t_t_max, t_s_max)

In [8]:
t_s_max

array([ 754,  301,  752,  915,  350,  761, 1234,  514,  635,  778,  644,
        420,  790,  614,  716,  582], dtype=int32)

In [ ]:
monotonic_align.maximum_path(log_prior, attn_mask)

In [15]:
speaker_id = None
checkpoint = "logs/new_exp/grad_183.pt"

In [16]:
if not isinstance(speaker_id, type(None)):
    assert params.n_spks > 1, "Ensure you set right number of speakers in `params.py`."
    spk = torch.LongTensor([speaker_id]).cuda()
else:
    spk = None

print('Initializing Grad-TTS...')
generator = GradTTS(len(symbols)+1, params.n_spks, params.spk_emb_dim,
                    params.n_enc_channels, params.filter_channels,
                    params.filter_channels_dp, params.n_heads, params.n_enc_layers,
                    params.enc_kernel, params.enc_dropout, params.window_size,
                    params.n_feats, params.dec_dim, params.beta_min, params.beta_max, params.pe_scale)
generator.load_state_dict(torch.load(checkpoint, map_location=lambda loc, storage: loc))
_ = generator.cuda().eval()
print(f'Number of parameters: {generator.nparams}')

print('Initializing HiFi-GAN...')
with open(HIFIGAN_CONFIG) as f:
    h = AttrDict(json.load(f))
vocoder = HiFiGAN(h)
vocoder.load_state_dict(torch.load(HIFIGAN_CHECKPT, map_location=lambda loc, storage: loc)['generator'])
_ = vocoder.cuda().eval()
vocoder.remove_weight_norm()

Initializing Grad-TTS...
Number of parameters: 14829464
Initializing HiFi-GAN...
Removing weight norm...


In [17]:
texts = ["По словам CEO и главного инженера SpaceX Илона Маска, Starship в будущем заменит Falcon 9 и Falcon Heavy, грузовой и пилотируемый Dragon V2[11].;"]

In [18]:
timesteps = 10

In [20]:
with torch.no_grad():
    for i, text in enumerate(texts):
        print(f'Synthesizing {i} text...', end=' ')
        x = torch.LongTensor(intersperse(text_to_sequence(text), len(symbols))).cuda()[None]
        x_lengths = torch.LongTensor([x.shape[-1]]).cuda()

        t = dt.datetime.now()
        y_enc, y_dec, attn = generator.forward(x, x_lengths, n_timesteps=timesteps, temperature=1.5,
                                               stoc=False, spk=spk, length_scale=0.91)
        t = (dt.datetime.now() - t).total_seconds()
        print(f'Grad-TTS RTF: {t * 22050 / (y_dec.shape[-1] * 256)}')

        audio = (vocoder.forward(y_dec).cpu().squeeze().clamp(-1, 1).numpy() * 32768).astype(np.int16)

        write(f'./out/sample_3.wav', 22050, audio)

Synthesizing 0 text... Grad-TTS RTF: 0.037381383895305514


In [8]:
from data import TextMelDataset, TextMelBatchCollate
from torch.utils.data import DataLoader

In [9]:
train_filelist_path = params.train_filelist_path
valid_filelist_path = params.valid_filelist_path
add_blank = params.add_blank

log_dir = params.log_dir
n_epochs = params.n_epochs
batch_size = params.batch_size
out_size = params.out_size
learning_rate = params.learning_rate
random_seed = params.seed

nsymbols = len(symbols) + 1 if add_blank else len(symbols)
n_enc_channels = params.n_enc_channels
filter_channels = params.filter_channels
filter_channels_dp = params.filter_channels_dp
n_enc_layers = params.n_enc_layers
enc_kernel = params.enc_kernel
enc_dropout = params.enc_dropout
n_heads = params.n_heads
window_size = params.window_size

n_feats = params.n_feats
n_fft = params.n_fft
sample_rate = params.sample_rate
hop_length = params.hop_length
win_length = params.win_length
f_min = params.f_min
f_max = params.f_max

dec_dim = params.dec_dim
beta_min = params.beta_min
beta_max = params.beta_max
pe_scale = params.pe_scale

In [10]:
train_dataset = TextMelDataset(
    train_filelist_path, add_blank, n_fft, n_feats, sample_rate, hop_length, win_length, f_min, f_max
)


KeyboardInterrupt



In [ ]:
batch_collate = TextMelBatchCollate()
loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                        collate_fn=batch_collate, drop_last=True,
                        num_workers=4, shuffle=False)

In [ ]:
%%timeit

for batch in loader:
    pass


In [ ]:
text = "По словам CEO и главного инженера SpaceX Илона Маска, Starship в будущем заменит Falcon 9 и Falcon Heavy, грузовой и пилотируемый Dragon V2[11].;"

In [ ]:
text_to_sequence(text, ["russian_cleaners"])

In [ ]:
with open(r"dataset/transcript.txt", "r", encoding="utf-8") as t:
    texts = t.read()

In [ ]:
texts

In [ ]:
a2 = [("dataset/" + i.split("|")[1], i.split("|")[1]) for i in texts.split("\n") if len(i.split("|")[:2]) == 2]

In [ ]:
test_p = 0.1
val_p = 0.05

max_c = 200


In [ ]:
a2

In [ ]:
dataset_path = r"dataset/"

data = []

for j in [i.split(",") for i in texts.split("\n") if len(i.split(",")) == 2]:
    with open(dataset_path + j[1], "r") as t:
        text = t.read()
        text = text.replace("\n", "")
        data.append((dataset_path + j[0], text))

In [ ]:
import random

In [ ]:
random.shuffle(a2)

test_c = int(min(len(a2) * test_p, max_c))
val_c = int(min(len(a2) * val_p, max_c))

test = a2[:test_c]
val = a2[test_c: test_c + val_c]
train = a2[test_c + val_c:]

In [ ]:
from data import TextMelDataset, TextMelBatchCollate
import params

In [ ]:
train_filelist_path = params.train_filelist_path
valid_filelist_path = params.valid_filelist_path
add_blank = params.add_blank

log_dir = params.log_dir
n_epochs = params.n_epochs
batch_size = params.batch_size
out_size = params.out_size
learning_rate = params.learning_rate
random_seed = params.seed

n_feats = params.n_feats
n_fft = params.n_fft
sample_rate = params.sample_rate
hop_length = params.hop_length
win_length = params.win_length
f_min = params.f_min
f_max = params.f_max

dec_dim = params.dec_dim
beta_min = params.beta_min
beta_max = params.beta_max
pe_scale = params.pe_scale

In [ ]:
train_dataset = TextMelDataset(
    train_filelist_path, add_blank, n_fft, n_feats, sample_rate, hop_length, win_length, f_min, f_max
)

In [ ]:
%%timeit
for batch in train_dataset:
    pass

In [ ]:
test_c

In [ ]:
a = normalizer(text)

In [19]:
a

'По словам се оу и главного инженера старши Илона Маска, п в будущем заменит фалкон девять и фалкон хеви грузовой и пилотируемый сругин,'

In [25]:
_whitespace_re = re.compile(r'[^a-zA-Z0-9А-Яа-яЁё]+')

In [27]:
a = " ".join(re.split(_whitespace_re, text))
a

'По словам CEO и главного инженера SpaceX Илона Маска Starship в будущем заменит Falcon 9 и Falcon Heavy грузовой и пилотируемый Dragon V2 11 '

In [28]:
normalizer(a)

'По словам се оу и главного инженера спасекс Илона Маска пятшип в будущем заменит фалкон девять и фалкон хеви грузовой и пилотируемый драгон в дв грузовой и пилотируемый в одиннадцать '

In [30]:
a = normalizer(a)
a

'По словам се оу и главного инженера спасекс Илона Маска пятшип в будущем заменит фалкон девять и фалкон хеви грузовой и пилотируемый драгон в дв грузовой и пилотируемый в одиннадцать '

In [32]:
a.split(" ")

['По',
 'словам',
 'се',
 'оу',
 'и',
 'главного',
 'инженера',
 'спасекс',
 'Илона',
 'Маска',
 'пятшип',
 'в',
 'будущем',
 'заменит',
 'фалкон',
 'девять',
 'и',
 'фалкон',
 'хеви',
 'грузовой',
 'и',
 'пилотируемый',
 'драгон',
 'в',
 'дв',
 'грузовой',
 'и',
 'пилотируемый',
 'в',
 'одиннадцать',
 '']

'PA SLAVAM S0I AU I GLAVNAVA INZHYN0IRA SPAS0IKS ILANA MASKA P0ITSHYP F BUDUSH0IM ZAM0IN0IT FALKAN D0IV0IT0 I FALKAN KH0IV0I GRUZAVAJ0 I P0ILAT0IRUJ0IMYJ0 DRAGAN F TF GRUZAVAJ0 I P0ILAT0IRUJ0IMYJ0 F AD0INATSAT0'

In [2]:
norm = Normalizer()

In [12]:
text = norm.norm_text(text)
print(text)

через сто тысяч лет жизнь на земле оборвётся и семь ноль ноль ноль ноль ноль пятьсот людей умрёт


In [14]:
text

In [7]:

a(text)

['S0',
 'I',
 'V',
 'A',
 'D',
 'N0',
 'A',
 'D',
 'V',
 'A',
 'TS',
 'A',
 'T0',
 'P0',
 'I',
 'T',
 'A',
 'V',
 'A',
 'I',
 'J0',
 'U',
 'N0',
 'A',
 'M',
 'Y',
 'D0',
 'I',
 'L',
 'A',
 'J0',
 'I',
 'M',
 'P',
 'R',
 'A',
 'G',
 'R',
 'A',
 'M',
 'U',
 'D0',
 'I',
 'P',
 'R0',
 'I',
 'K',
 'A',
 'SH',
 'Y',
 'N',
 'V',
 'A',
 'R',
 'N0',
 'I',
 'N',
 'G',
 'D',
 'V',
 'A']